In [1]:
from project_huishoudboekje.config import GeneralSettings
from project_huishoudboekje.read_rabo import ReadRabo

import os
import pandas as pd

In [4]:
df = pd.read_csv(GeneralSettings.project_path / 'data/Rabobank/CSV_A_NL45RABO0372430163_EUR_202207.csv', encoding='latin-1')

In [5]:
df.head()

,IBAN/BBAN,Munt,BIC,Volgnr,Datum,Rentedatum,Bedrag,Saldo na trn,Tegenrekening IBAN/BBAN,Naam tegenpartij,...,Machtigingskenmerk,Incassant ID,Betalingskenmerk,Omschrijving-1,Omschrijving-2,Omschrijving-3,Reden retour,Oorspr bedrag,Oorspr munt,Koers
0,NL45RABO0372430163,EUR,RABONL2U,6749,2022-07-01,2022-07-01,"-21,66","+5374,79",NL24RABO0168476207,Basic Fit Nederland B.V.,...,305012750,NL47ZZZ370924590000,NaN,01-07-2022 / 31-07-2022,,NaN,NaN,NaN,NaN,NaN
1,NL45RABO0372430163,EUR,RABONL2U,6750,2022-07-01,2022-07-01,"-10,00","+5364,79",NL02RABO0387512012,ZOA,...,ZOA201706557823,NL39ZZZ410097230000,NaN,Hartelijk dank voor uw bijdrage.,,NaN,NaN,NaN,NaN,NaN
2,NL45RABO0372430163,EUR,RABONL2U,6751,2022-07-01,2022-07-01,"+80,00","+5444,79",NL18ABNA0481361669,SERVICEBUREAU JANSEN BV,...,NaN,NaN,NaN,Philips Personal Health - Reg ID 3356177,,NaN,NaN,NaN,NaN,NaN
3,NL45RABO0372430163,EUR,RABONL2U,6752,2022-07-01,2022-07-01,"-7,70","+5437,09",NaN,Hoogvliet 700 5 AMERSFOORT,...,NaN,NaN,NaN,Betaalautomaat 09:16 pasnr. 038 Apple Pay,,NaN,NaN,NaN,NaN,NaN
4,NL45RABO0372430163,EUR,RABONL2U,6753,2022-07-01,2022-07-01,"-16,96","+5420,13",NL15DEUT0319871975,NATIONALE-NEDERLANDEN VERZEKEREN,...,4379705V1,NL34ZZZ664133540000,NaN,INCASSO ZEKERHEIDSPAKKET PARTICULIEREN POLIS 1...,,NaN,NaN,NaN,NaN,NaN


In [6]:
df['DATE'] = pd.to_datetime(df['Datum'], format='%Y-%m-%d')

In [20]:
df['AMOUNT_NO_ABS'] = df['Bedrag'].apply(lambda x: float(x.replace('+', '').replace(',', '.')))

In [21]:
df['FINANCIAL_TYPE'] = df['AMOUNT_NO_ABS'].apply(lambda x: 'credit' if x > 0 else 'debet')

In [22]:
df['AMOUNT'] = df['AMOUNT_NO_ABS'].abs()

In [23]:
df['ANALYSE_IND'] = df['Code'].apply(lambda x: 0 if x == 'tb' else 1)

In [24]:
df['PARTY'] = df['Naam tegenpartij'].astype(str) + ' - ' + df['Omschrijving-1'].str.split().str[:3].str.join(sep=" ")

In [ ]:
df.rename(columns={'Code': 'TRANSACTION_TYPE'})

In [2]:
ReadRabo().run(['CSV_A_NL45RABO0372430163_EUR_202207.csv'])

,DATE,FINANCIAL_TYPE,TRANSACTION_TYPE,PARTY,AMOUNT,ANALYSE_IND
0,2022-07-01,debet,ei,Basic Fit Nederland B.V. - 01-07-2022 / 31-07-...,21.66,1
1,2022-07-01,debet,ei,ZOA - Hartelijk dank voor,10.00,1
2,2022-07-01,credit,cb,SERVICEBUREAU JANSEN BV - Philips Personal Health,80.00,1
3,2022-07-01,debet,bc,Hoogvliet 700 5 AMERSFOORT - Betaalautomaat 09...,7.70,1
4,2022-07-01,debet,ei,NATIONALE-NEDERLANDEN VERZEKEREN - INCASSO ZEK...,16.96,1
...,...,...,...,...,...,...
90,2022-07-29,debet,bc,CP CASCAIS 2750-340-CASC PRT - Betaalautomaat ...,10.20,1
91,2022-07-29,debet,bc,REST A5 DO PACO 1100-148 LISB PRT - Betaalauto...,9.60,1
92,2022-07-30,debet,bc,LIDL AGRADECE CASCAIS PRT - Betaalautomaat 12:...,14.59,1
93,2022-07-30,debet,bc,REST VANILLA LEADER CASCAIS PRT - Betaalautoma...,15.00,1
